<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/notebook/0_Learn%20the%20Basics/0_8_quickstart_tutorial_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

「PyTorch入門  8. クイックスタート」
===============================================================
【原題】Learn the Basics

【原著】
[Suraj Subramanian](https://github.com/suraj813)、[Seth Juarez](https://github.com/sethjuarez/) 、[Cassie Breviu](https://github.com/cassieview/) 、[Dmitry Soshnikov](https://soshnikov.com/)、[Ari Bornstein](https://github.com/aribornstein/)


【元URL】https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

【翻訳】電通国際情報サービスISID AIトランスフォーメーションセンター　小川 雄太郎

【日付】2021年03月20日

【チュトーリアル概要】

本チュートリアルでは、機械学習を実行するためのPyTorchの各APIを紹介します。詳細を知りたい方は、各セクションのリンク先をご参照ください。

---


【注意】

ディープラーニングフレームワークの経験があり、ディープラーニングの実装に慣れている方は、本チュートリアルをまず確認し、PyTorchのAPIに慣れてください。

<br>

ディープラーニングフレームワークを利用した実装に初めて取り組む方は、本チュートリアルからではなく、

[「PyTorch入門  1. テンソル」](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/1_1_tensor_tutorial_jp.ipynb)から、1 stepずつ進めてください。



---


データの取り扱い
-----------------

PyTorchではデータを取り扱う際に、基本的な要素が2つ存在します。

``torch.utils.data.Dataset``と``torch.utils.data.DataLoader``です。

<br>

``Dataset`` は各ラベルと、それに対応するサンプルデータを保持します。

``DataLoader`` は``Dataset``をイテレーティブに（＝反復処理可能に）操作できるようラップしたものになります。

<br>

（詳細）
https://pytorch.org/docs/stable/data.html





In [ ]:
%matplotlib inline

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorchには以下に示すようなドメイン固有のライブラリが存在し、それぞれにデータセットが用意されています。


本チュートリアルでは、TorchVisionのデータセットを使用します。

- [TorchText](https://pytorch.org/text/stable/index.html)
- [TorchVision](https://pytorch.org/vision/stable/index.html)
- [TorchAudio](https://pytorch.org/audio/stable/index.html)


``torchvision.datasets`` モジュールには、画像データの ``Dataset`` オブジェクトがたくさん用意されています。

例えば、CIFAR, COCOなどです ([データセット一覧はこちらから](https://pytorch.org/docs/stable/torchvision/datasets.html))。

本チュートリアルでは、FashionMNISTデータセットを使用します。

TorchVisionの すべての``Dataset`` には2つの引数があります。

``transform`` と ``target_transform`` であり、それぞれサンプルとラベルの変換処理を行います。

In [ ]:
# 訓練データをdatasetsからダウンロード
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# テストデータをdatasetsからダウンロード
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

DataLoaderの引数としてDatasetを渡します。


DataLoaderはDatasetをイテレート処理できるようにラップしたものです。

バッチ処理、サンプリング、シャッフル、マルチプロセスでのデータ読み込み等をサポートしています。




以下では、バッチサイズを64と定義します。

つまり、データローダの反復要素は、64個の特徴量とラベルから構成されるバッチを返すことになります。

In [ ]:
batch_size = 64

# データローダーの作成
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

さらなる詳細は、[「PyTorch入門  2. データセットとデータローダー」](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/0_2_data_tutorial_jp.ipynb
)をご覧ください。


--------------




モデルの構築
------------------

PyTorchでニューラルネットワークの形を定義する際には、[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)を継承します。


``__init__`` 関数で、ネットワークの各レイヤーを定義し、データの順伝搬を``forward`` 関数に定義します。

なお処理を高速化するために、可能であればニューラルネットワークをGPU上へ移動させます。



In [ ]:
# 訓練に際して、可能であればGPU（cuda）を設定します。GPUが搭載されていない場合はCPUを使用します
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# modelを定義します
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


さらなる詳細は、[「PyTorch入門 4. モデル構築」](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/0_4_buildmodel_tutorial_js.ipynb
)をご覧ください。



--------------




モデルパラメータの最適化
----------------------------------------

ニューラルネットワークモデルを訓練するためには、

損失関数：[loss function](<https://pytorch.org/docs/stable/nn.html#loss-functions)と、最適化手法：[optimizer](https://pytorch.org/docs/stable/optim.html)が必要です。



In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

1回の訓練ループにおいてモデルは、まず訓練データのバッチに対して推論を実行して損失誤差を計算し、

その後、損失誤差をバックプロパゲーションして、モデルパラメータを更新します。




In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 損失誤差を計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

また、モデルがうまく学習していることを確認するために、テストデータセットに対するモデルの性能も確認します。



In [ ]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

訓練プロセスでは複数イテレーションを実行します。

各エポックの間、モデルはより良い予測ができるようにパラメータを学習します。

エポックごとにモデルの正解率と損失を出力して、正解率が向上し、損失が低下していっているかを確認します。




In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

さらなる詳細は、[「PyTorch入門 6. 最適化」](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/0_6_optimization_tutorial_js.ipynb)をご覧ください。


--------------




モデルの保存
-------------

モデルを保存する一般的な方法は、モデルの内部状態の辞書（モデルのパラメータを含む）をシリアル化する手法です。


In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

---


モデルの読み込み
----------------------------

モデルの読み込む際には、まずモデルの構造を再作成し、そのインスタンスに、保存しておいた状態辞書をロードします。


In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

これでモデルは推論可能な状態です。




In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

さらなる詳細は、[「PyTorch入門 7. モデルの保存・読み込み」](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/0_7_saveloadrun_tutorial_js.ipynb)をご覧ください。





以上。